# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lr_project').getOrCreate()

In [4]:
data = spark.read.csv("/FileStore/tables/cruise_ship_info.csv",inferSchema=True,header=True)

In [5]:
data.show(3)

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
 Ship_name|Cruise_line|Age| Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
 Journey| Azamara| 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55|
 Quest| Azamara| 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55|
Celebration| Carnival| 26| 47.262| 14.86| 7.22| 7.43| 31.8| 6.7|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
only showing top 3 rows

In [6]:
data.describe().show()

+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
summary|Ship_name|Cruise_line| Age| Tonnage| passengers| length| cabins|passenger_density| crew|
+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
 count| 158| 158| 158| 158| 158| 158| 158| 158| 158|
 mean| Infinity| null|15.689873417721518| 71.28467088607599|18.45740506329114|8.130632911392404| 8.830000000000005|39.90094936708861|7.794177215189873|
 stddev| NaN| null| 7.615691058751413|37.229540025907866|9.677094775143416|1.793473548054825|4.4714172221480615| 8.63921711391542|3.503486564627034|
 min|Adventure| Azamara| 4| 2.329| 0.66| 2.79| 0.33| 17.7| 0.59|
 max|Zuiderdam| Windstar| 48| 220.0| 54.0| 11.82| 27.0| 71.43| 21.0|
+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+

In [7]:
from pyspark.sql.functions import countDistinct

data.select(countDistinct('Ship_name'),countDistinct('Cruise_line')).show()

+-------------------------+---------------------------+
count(DISTINCT Ship_name)|count(DISTINCT Cruise_line)|
+-------------------------+---------------------------+
 138| 20|
+-------------------------+---------------------------+

In [8]:
data.groupBy('Cruise_line').count().show()

+-----------------+-----+
 Cruise_line|count|
+-----------------+-----+
 Costa| 11|
 P&O| 6|
 Cunard| 3|
Regent_Seven_Seas| 5|
 MSC| 8|
 Carnival| 22|
 Crystal| 2|
 Orient| 1|
 Princess| 17|
 Silversea| 4|
 Seabourn| 3|
 Holland_American| 14|
 Windstar| 3|
 Disney| 2|
 Norwegian| 13|
 Oceania| 3|
 Azamara| 2|
 Celebrity| 10|
 Star| 6|
 Royal_Caribbean| 23|
+-----------------+-----+

In [9]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="Cruise_line", outputCol = 'Cruise_line_index',handleInvalid='keep', stringOrderType='frequencyDesc')

result = indexer.fit(data).transform(data)

indexer02 = StringIndexer(inputCol="Ship_name", outputCol = 'Ship_name_index',handleInvalid='keep', stringOrderType='frequencyDesc')

result = indexer02.fit(result).transform(result)

result = result.drop('Ship_name','Cruise_line')

result.show()

+---+------------------+----------+------+------+-----------------+----+-----------------+---------------+
Age| Tonnage|passengers|length|cabins|passenger_density|crew|Cruise_line_index|Ship_name_index|
+---+------------------+----------+------+------+-----------------+----+-----------------+---------------+
 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55| 16.0| 64.0|
 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55| 16.0| 98.0|
 26| 47.262| 14.86| 7.22| 7.43| 31.8| 6.7| 1.0| 27.0|
 11| 110.0| 29.74| 9.53| 14.88| 36.99|19.1| 1.0| 31.0|
 17| 101.353| 26.42| 8.92| 13.21| 38.36|10.0| 1.0| 34.0|
 22| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0| 37.0|
 15| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0| 38.0|
 23| 70.367| 20.56| 8.55| 10.22| 34.23| 9.2| 1.0| 47.0|
 19| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0| 48.0|
 6|110.23899999999999| 37.0| 9.51| 14.87| 29.79|11.5| 1.0| 5.0|
 10| 110.0| 29.74| 9.51| 14.87| 36.99|11.6| 1.0| 53.0|
 28| 46.052| 14.52| 7.27| 7.26| 31.72| 6.6| 1.0| 57.0|
 18| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0| 58.0|
 17| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0| 62.0|
 11| 86.0| 21.24| 9.63| 10.62| 40.49| 9.3| 1.0| 1.0|
 8| 110.0| 29.74| 9.51| 14.87| 36.99|11.6| 1.0| 66.0|
 9| 88.5| 21.24| 9.63| 10.62| 41.67|10.3| 1.0| 78.0|
 15| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0| 89.0|
 12| 88.5| 21.24| 9.63| 11.62| 41.67| 9.3| 1.0| 10.0|
 20| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0| 107.0|
+---+------------------+----------+------+------+-----------------+----+-----------------+---------------+
only showing top 20 rows

In [10]:
result.printSchema()

result.columns

root
-- Age: integer (nullable = true)
-- Tonnage: double (nullable = true)
-- passengers: double (nullable = true)
-- length: double (nullable = true)
-- cabins: double (nullable = true)
-- passenger_density: double (nullable = true)
-- crew: double (nullable = true)
-- Cruise_line_index: double (nullable = false)
-- Ship_name_index: double (nullable = false)

Out[46]: ['Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Cruise_line_index',
 'Ship_name_index']

## Machine Learning

In [12]:
# Import VectorAssembler and Vectors
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [13]:
assembler = VectorAssembler(
    inputCols=['Age',
               'Tonnage',
               'passengers',
               'length',
               'cabins',
               'passenger_density',
               'Cruise_line_index',
               'Ship_name_index'],
    outputCol="features")

In [14]:
output = assembler.transform(result)

In [15]:
output.head(1)

Out[50]: [Row(Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, Cruise_line_index=16.0, Ship_name_index=64.0, features=DenseVector([6.0, 30.277, 6.94, 5.94, 3.55, 42.64, 16.0, 64.0]))]

In [16]:
# Selecionando as variáveis independentes e a variável que queremos prever

final_data = output.select('features','crew')

In [17]:
# Amostra Holdout

train_data,test_data = final_data.randomSplit([0.7,0.3])

In [18]:
from pyspark.ml.regression import LinearRegression

# Create a Linear Regression Model object
lr = LinearRegression(labelCol='crew', featuresCol='features', maxIter=1000)

# Fit the model to the data and call this model lrModel
lrModel = lr.fit(train_data)

In [19]:
# Print the coefficients and intercept for linear regression
print("Coefficients: {} \n Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

Coefficients: [-0.011103671270742367,0.012619485950495644,-0.1587483475718366,0.3116432891246372,0.8969159631630635,-0.009127031114345631,0.03970071650879419,0.0006690423388478407] 
 Intercept: -0.3493284660349399

In [20]:
# Evaluate the train model

test_results = lrModel.evaluate(test_data)

print("RMSE:    {}".format(test_results.rootMeanSquaredError))
print("MSE:     {}".format(test_results.meanSquaredError))
print('MAE:     {}'.format(test_results.meanAbsoluteError))
print('Adj. R2: {}'.format(test_results.r2adj))
print('R2:      %f' % (test_results.r2))

RMSE: 1.0477148925476076
MSE: 1.0977064960660452
MAE: 0.7232333541723093
Adj. R2: 0.878241287738439
R2: 0.901999

In [21]:
# Showing the real data (crew) vs predicted data (prediction)

test_results.predictions.show()

+--------------------+-----+------------------+
 features| crew| prediction|
+--------------------+-----+------------------+
[5.0,160.0,36.34,...| 13.6|15.245402722696323|
[6.0,90.0,20.0,9....| 9.0| 9.908932163386124|
[6.0,93.0,23.94,9...|11.09|10.539006944505642|
[8.0,91.0,22.44,9...| 11.0|10.011373320906083|
[9.0,110.0,29.74,...| 11.6|12.306074618265214|
[10.0,46.0,7.0,6....| 4.47| 2.535926375377585|
[10.0,58.825,15.6...| 7.0| 7.210952799149331|
[10.0,68.0,10.8,7...| 6.36| 6.252166452560552|
[10.0,77.0,20.16,...| 9.0| 8.74880971842105|
[10.0,81.76899999...| 8.42| 8.686110368666807|
[10.0,138.0,31.14...|11.85|13.082285754282227|
[10.0,151.4,26.2,...|12.53| 11.00303287747748|
[11.0,86.0,21.24,...| 9.3| 9.439180711393488|
[11.0,90.0,22.4,9...| 11.0| 9.955893352617583|
[11.0,91.0,20.32,...| 9.99| 9.054331530310396|
[11.0,138.0,31.14...|11.85|13.071851125350332|
[12.0,2.329,0.94,...| 0.6|0.8253592754545256|
[12.0,108.865,27....| 11.0|10.891889392612883|
[13.0,25.0,3.82,5...| 2.95|2.7295270678687364|
[13.0,91.0,20.32,...| 9.99| 9.062231093017063|
+--------------------+-----+------------------+
only showing top 20 rows

In [22]:
# R2 of 0.92 is pretty good, let's check the data a little closer
from pyspark.sql.functions import corr

data.select(corr('crew','passengers')).show()

+----------------------+
corr(crew, passengers)|
+----------------------+
 0.9152341306065384|
+----------------------+

In [23]:
final_data.describe().show()

+-------+-----------------+
summary| crew|
+-------+-----------------+
 count| 158|
 mean|7.794177215189873|
 stddev|3.503486564627034|
 min| 0.59|
 max| 21.0|
+-------+-----------------+

In [24]:
# Deployment

unlabeled_data = test_data.select('features')

predictions = lrModel.transform(unlabeled_data)

predictions.show()

+--------------------+------------------+
 features| prediction|
+--------------------+------------------+
[5.0,160.0,36.34,...|15.245402722696323|
[6.0,90.0,20.0,9....| 9.908932163386124|
[6.0,93.0,23.94,9...|10.539006944505642|
[8.0,91.0,22.44,9...|10.011373320906083|
[9.0,110.0,29.74,...|12.306074618265214|
[10.0,46.0,7.0,6....| 2.535926375377585|
[10.0,58.825,15.6...| 7.210952799149331|
[10.0,68.0,10.8,7...| 6.252166452560552|
[10.0,77.0,20.16,...| 8.74880971842105|
[10.0,81.76899999...| 8.686110368666807|
[10.0,138.0,31.14...|13.082285754282227|
[10.0,151.4,26.2,...| 11.00303287747748|
[11.0,86.0,21.24,...| 9.439180711393488|
[11.0,90.0,22.4,9...| 9.955893352617583|
[11.0,91.0,20.32,...| 9.054331530310396|
[11.0,138.0,31.14...|13.071851125350332|
[12.0,2.329,0.94,...|0.8253592754545256|
[12.0,108.865,27....|10.891889392612883|
[13.0,25.0,3.82,5...|2.7295270678687364|
[13.0,91.0,20.32,...| 9.062231093017063|
+--------------------+------------------+
only showing top 20 rows